In [1]:
import numpy as np
import pandas as pd
import networkx as nx

import altair as alt

/home/pi/code/envs/graph/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Create Graphs

In [2]:
G = nx.karate_club_graph()
pos = nx.layout.spring_layout(G)

### Embue arbitrary data

In [3]:
wordlist = "Declarative visualization grammars can accelerate development, facilitate retargeting across platforms, and allow language-level optimizations. However, existing declarative visualization languages are primarily concerned with visual encoding, and rely on imperative event handlers for interactive behaviors. In response, we introduce a model of declarative interaction design for data visualizations. Adopting methods from reactive programming, we model low-level events as composable data streams from which we form higher-level semantic signals. Signals feed predicates and scale inversions, which allow us to generalize interactive selections at the level of item geometry (pixels) into interactive queries over the data domain. Production rules then use these queries to manipulate the visualization’s appearance. To facilitate reuse and sharing, these constructs can be encapsulated as named interactors: standalone, purely declarative specifications of interaction techniques. We assess our model’s feasibility and expressivity by instantiating it with extensions to the Vega visualization grammar. Through a diverse range of examples, we demonstrate coverage over an established taxonomy of visualization interaction techniques.We present Reactive Vega, a system architecture that provides the first robust and comprehensive treatment of declarative visual and interaction design for data visualization. Starting from a single declarative specification, Reactive Vega constructs a dataflow graph in which input data, scene graph elements, and interaction events are all treated as first-class streaming data sources. To support expressive interactive visualizations that may involve time-varying scalar, relational, or hierarchical data, Reactive Vega’s dataflow graph can dynamically re-write itself at runtime by extending or pruning branches in a data-driven fashion. We discuss both compile- and run-time optimizations applied within Reactive Vega, and share the results of benchmark studies that indicate superior interactive performance to both D3 and the original, non-reactive Vega system.We present Vega-Lite, a high-level grammar that enables rapid specification of interactive data visualizations. Vega-Lite combines a traditional grammar of graphics, providing visual encoding rules and a composition algebra for layered and multi-view displays, with a novel grammar of interaction. Users specify interactive semantics by composing selections. In Vega-Lite, a selection is an abstraction that defines input event processing, points of interest, and a predicate function for inclusion testing. Selections parameterize visual encodings by serving as input data, defining scale extents, or by driving conditional logic. The Vega-Lite compiler automatically synthesizes requisite data flow and event handling logic, which users can override for further customization. In contrast to existing reactive specifications, Vega-Lite selections decompose an interaction design into concise, enumerable semantic units. We evaluate Vega-Lite through a range of examples, demonstrating succinct specification of both customized interaction methods and common techniques such as panning, zooming, and linked selection.".split(' ')
for edge in G.edges:
    G.edges[edge]['words'] = np.random.choice(wordlist, size=np.random.randint(8), replace=False)

for node in G.nodes:
    G.nodes[node]['favorite_color'] =  np.random.choice(['blue', 'blue', 'blue', 'yellow'], replace=False)

### Calculate Communities

In [4]:
from networkx.algorithms.community import greedy_modularity_communities

In [5]:
for node in G.nodes:
    for i,v in enumerate(greedy_modularity_communities(G)):
        if node in v: G.nodes[node]['community'] = i

In [6]:
def _add_position(row, pos):
    def get_pos_data(pos):
        return pd.DataFrame.from_dict(pos).transpose().reset_index().rename(
            columns={'index': 'node_id', 0:'x', 1:'y'}
        )
    def pos_lookup(node_id, pos_data):
        return pos_data.query(f'node_id=={node_id}').values[0]
    pos_data = get_pos_data(pos)
    _, row['x'], row['y'] = pos_lookup(row.node_id, pos_data)
    return row

def alt_node_data(G, pos):
    node_data_flat = pd.DataFrame.from_dict(G.nodes.data())
    return node_data_flat.join(
        pd.json_normalize(node_data_flat.pop(1))
    ).rename(columns={0:'node_id'}).apply(
        _add_position, axis=1, args=[pos]
    )

def alt_edge_data(G, pos):
    edge_data = nx.to_pandas_edgelist(G).reset_index(
    ).rename(
        columns={'index': 'edge_id'}
    )
    id_vars = [c for c in edge_data.columns if c not in ['source', 'target']]
    melted_edge_data = edge_data.melt(id_vars=id_vars, var_name='end', value_name='node_id')
    return melted_edge_data.apply(
        _add_position, axis=1, args=[pos]
    )

### Convert nx nodes, edges to altair format

In [7]:
node_data = alt_node_data(G, pos)
edge_data = alt_edge_data(G, pos)

In [8]:
nodes = alt.Chart(node_data).mark_point(
    size=200, 
).encode(
    x='x:Q',
    y='y:Q',
    tooltip=['node_id', 'club'],
    fill='community:N'
)

labels = nodes.mark_text(
    align='left',
    baseline='middle',
    dx=10,
    dy=-10
).encode(text='node_id')

edges = alt.Chart(edge_data).mark_line(    
).encode(
    x = alt.X('x:Q', axis=None),
    y = alt.Y('y:Q', axis=None),
    detail='edge_id',
    strokeWidth='n_words:Q'
)

drawing = (
    nodes + labels + edges
).configure_view(strokeWidth=0)
drawing

alt.LayerChart(...)